In [168]:
import random
import networkx as nx
import itertools
import time

In [169]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [170]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [171]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [172]:
def fitness(resolving_set,G,shortest_paths,all_pairs):
    unresolved_pairs = 0
    for u, v in all_pairs:
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1

    return len(resolving_set) + 50 * unresolved_pairs 

In [173]:
def construct_solution1(G,pheromones,heuristics,alpha,beta,shortest_paths,all_pairs):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:
        # Računanje vjerovatnoće izbora svakog čvora
        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}
        total = sum(probabilities.values())
        
        if total == 0:
            break
        
        # Normalizacija vjerovatnoća
        probabilities = {node: prob / total for node, prob in probabilities.items()}

      
        selected_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        # Ako je rješenje dovoljno dobro, prekidamo ranije
        if fitness(resolving_set,G,shortest_paths,all_pairs) < 100:
            break

    return resolving_set

In [174]:
def construct_solution2(G, pheromones,heuristics, alpha, beta, shortest_paths, all_pairs):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:

        G_sub = G.subgraph(available_nodes)

        heuristics = nx.betweenness_centrality(G_sub, normalized=True)

        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}

        total = sum(probabilities.values())

        if total == 0:
            break

        probabilities = {node: w / total for node, w in probabilities.items()}

        selected_node = random.choices(population=list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        if fitness(resolving_set, G, shortest_paths, all_pairs) < 100:
            break

    return resolving_set

In [175]:
def update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths,all_pairs):
    for node in pheromones:
        pheromones[node] *= (1 - evaporation)

    for node in best_solution:
        pheromones[node] += Q / fitness(best_solution,G,shortest_paths,all_pairs)

In [176]:
def ant_colony_optimization(G,pheromones,heuristics,shortest_paths,all_pairs,num_iterations,num_ants,alpha,beta,evaporation,Q,stop):
    best_solution = None
    best_fitness = float("inf")
    no_improvement = 0

    for iteration in range(num_iterations):
        solutions = [construct_solution2(G,pheromones,heuristics,alpha,beta,shortest_paths,all_pairs) for _ in range(num_ants)]
        
        sorted_solutions = sorted(solutions, key=lambda s: fitness(s, G,shortest_paths,all_pairs))        
        if fitness(sorted_solutions[0],G,shortest_paths,all_pairs) < best_fitness:
            best_solution = sorted_solutions[0]
            best_fitness = fitness(best_solution,G,shortest_paths,all_pairs)
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= stop:
            print(f"Prekid: nema poboljšanja tokom {stop} generacija.")
            break

        update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths,all_pairs)
        
        print(f"Iteracija {iteration}, najbolji skup {best_fitness},  {best_solution}")
    print("\nOptimalni rješavajući skup:", best_solution, 'velicine:',len(best_solution))
    return best_solution,len(best_solution)

In [177]:
G = load_graph('grafovi\csp\csp100.txt')

if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()

n = len(G.nodes)
centrality = nx.betweenness_centrality(G)
# heuristics = {node: 1.0 / (sum(nx.shortest_path_length(G, node, j) for j in G.nodes if j != node) + 1) for node in G.nodes}
heuristics = {node: centrality[node] for node in G.nodes}
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
all_pairs = list(itertools.combinations(G.nodes, 2))
pheromones = {node: 1.0 for node in G.nodes}

Graf nije povezan!


<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_457020\472906638.py:1: SyntaxWarning: invalid escape sequence '\c'
  G = load_graph('grafovi\csp\csp100.txt')


In [178]:
param_grid = {
    'alpha': [1],              # manji fokus na feromone — više istraživanja
    'beta': [3],               # umjerena važnost heuristike (npr. betweenness)
    'evaporation': [0.3],         # dosta brzo isparavanje — algoritam brzo “zaboravlja”
    'Q': [50],                # visoke vrijednosti → jači doprinos dobrih rješenja
    'num_ants': [2],         # dovoljno da se pokrije prostor pretrage
    'num_iterations': [30],       # relativno mali broj iteracija — štedi vrijeme
    'stop': [6]                # rani prekid ako se ne vidi napredak
}

best_score = float('inf')
best_params = None

for combo in itertools.product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), combo))
    
    print(f"Testing: {params}")
    
    start_time = time.time()  
    os, score = ant_colony_optimization(G,pheromones,heuristics,shortest_paths,all_pairs,**params)
    end_time = time.time()    
    
    duration = end_time - start_time
    print(f"🔁 Rezultat: {score} | ⏱️ Vrijeme: {duration:.2f} sekundi",os)
    
    if (score < best_score) or (score == best_score and duration < duration1):
        if(is_resolving_set(os,G)):
            best_score = score
            best_params = params
            duration1 = duration

print("\n✅ Najbolji rezultat:", best_score)
print("📌 Najbolji parametri:", best_params)
print("📌 Vrijeme:", duration1)

Testing: {'alpha': 1, 'beta': 3, 'evaporation': 0.3, 'Q': 50, 'num_ants': 2, 'num_iterations': 30, 'stop': 6}
Iteracija 0, najbolji skup 85,  {20, 25, 27, 28, 31, 36, 37, 39, 40, 42, 43, 45, 46, 48, 49, 50, 51, 52, 57, 58, 59, 61, 67, 69, 70, 71, 72, 73, 81, 82, 83, 84, 86, 89, 92}
Iteracija 1, najbolji skup 85,  {20, 25, 27, 28, 31, 36, 37, 39, 40, 42, 43, 45, 46, 48, 49, 50, 51, 52, 57, 58, 59, 61, 67, 69, 70, 71, 72, 73, 81, 82, 83, 84, 86, 89, 92}


KeyboardInterrupt: 

In [ ]:
is_resolving_set(os,G)

False